In [ ]:
'''
:@Author: Remi
:@Date: 2023/9/13 08:21:53
:@LastEditors: Remi
:@LastEditTime: 2023/10/19 20:07:26
:Description: 
'''
import numpy as np
import preprocessing as pp
import file_tool as ft
import math

path = r'D:\20231017\origin'
path_list = ft.find_file(path,'.csv')
name_list = ['1000','100','10','500','50','300','30']
n = 0

for paths in path_list:
    wl,spec = ft.read_ava(paths)
    slice = 0
    spec = pp.Anomalous_spectrum_removal(spec[slice:,:])
    wl = wl[slice:]#################################################################切片了，不切的删掉这行
    spec_num = len(spec[0,:])
    rsd = np.array([pp.RSD_calculate(spec[wave,:],int(spec_num/10)) for wave in range(len(wl))])
    origin_spec = spec
   
    ##############genrate slope map#########
    ###################feather devide###########    


    

    for i in range(spec_num):#calculate slope
        origin_mean_spec = np.mean(spec,axis = 1)
        spec[:,i] = pp.Normalization(spec[:,i],method='total_intensity')
        
        single_spec = spec[:,i]
        slope = np.int64((np.append(single_spec,single_spec[-1]) - np.insert(single_spec,0,single_spec[0]))>= 0)#eg  0123 vs 0 -1 1 1 0 ,-1 is the slope between num0&1
        peak = np.insert(slope,0,slope[0]) - np.append(slope,slope[-1])
        if i == 0:
            slope_map = slope
            peak_map = peak
        else:
            slope_map = np.vstack((slope_map,slope)) 
            peak_map = np.vstack((peak_map,peak)) 
            #map 1 = + while 0 = -
    
    slope_map[slope_map == 0] = - 1
    peak_map = peak_map[:,1:-1].T###计算peak形状

    for single_ in slope_map:###计算 arise 和 decay 形状
        for i in range(len(single_)-1):
            if single_[i] != single_[i+1]:
                single_[i] = 0

    scores =  (np.abs(np.sum(peak_map,axis = 1)) +  np.abs(np.sum(slope_map[:,:-1],axis = 0)))/spec_num
    mean_spec = np.mean(spec,axis = 1)


    '''
    different weighed method
    '''
    #weighter_spec = mean_spec * np.log((scores * (math.e - 1) + 1))
    weighter_spec = mean_spec * scores
    
    ######################这部分先不用了###################
    #slice_wl,slice_spec = pp.spec_slice([50,50],[50,50],[11544,11644],wl,spec)
    #slice_wl = slice_wl.flatten()

    #np.savetxt(path + '//' + name_list[n] + 'rsd.csv',np.hstack((wl,np.array(([rsd])).T)),delimiter=',',fmt='%.04f')
    np.savetxt(path + '//' + name_list[n] + 'origin_mean.csv',np.hstack((wl,np.array(([origin_mean_spec])).T)),delimiter=',',fmt='%.04f')
    #np.savetxt(path + '//' + name_list[n] + 'normalized.csv',np.hstack((wl,spec)),delimiter=',',fmt='%.04f')
    #np.savetxt(path + '//' + name_list[n] + 'origin.csv',np.hstack((wl,origin_spec)),delimiter=',',fmt='%.04f')
    np.savetxt(path + '//' + name_list[n] + 'weighted.csv',np.hstack((wl,np.vstack((mean_spec,weighter_spec)).T)),delimiter=',',fmt='%.04f')
    n = n+1

